# 4. Introducing the Grid Network


In the previous tutorials we explicitly connected to the workers in order to interact with them, this can be useful for use cases where we want to have control over where the data is stored (which node?) and who performs each role on the application.

Another possible way to see Grid is that workers that are connected consist of a Grid Network.


## What is the Grid Network?

<p align="center">
<img height="600px" width="600px" src="https://github.com/OpenMined/rfcs/blob/master/20190821-grid-platform/DHT-grid.png?raw=true">
</p>


This network can be seen by a user as a single interface: the Grid Gateway. The Gateway works like a special DNS component but it will route nodes by queries instead of domain names.

It is important to emphasize: the Grid Gateway will not be able to perform any computation process on the nodes. It can not concentrate or centralize any data or model. It works as a brigde between a user that is outside of the grid network to get data / models that live inside the network.


<p align="center">
<img height="600px" width="600px" src="https://github.com/OpenMined/rfcs/blob/master/20190821-grid-platform/partially_grid.png?raw=true">
</p>

With this fully distributed and descentralized architecure we can provide fault tolerance in a very transparent way.


## Using the Grid Network


In this notebook we'll use a grid network to host a toy model.

In [18]:
import torch as th
import grid as gr
import syft as sy
import torch.nn.functional as F


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

hook = sy.TorchHook(th)

## Connect to the Grid Network

We don't need to connect to each worker explictly we just connect to a gateway.

In [6]:
# Checkout the example folders to learn how to setup a GridNetwork on Heroku!
grid_network = gr.GridNetwork("http://opengridgateway.herokuapp.com")

In [11]:
# Build Model
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = th.nn.Linear(2, 1)
        self.fc2 = th.nn.Linear(1, 1)
        self.state += ["fc1", "fc2"]

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()
model.build(th.tensor([1.0, 2]))

In [12]:
grid_network.serve_model(model, model_id="toy_model",allow_remote_inference=True, allow_download=False)

In [13]:
grid_network.run_remote_inference(model_id="toy_model", data=th.tensor([1.0, 2]))

tensor([0.7818])

In [15]:
worker = grid_network.query_model(model_id="toy_model")

print("This model was hosted at: ", worker.id)
print("Grid Address: ", worker.uri)


This model was hosted at:  hospitalnode
Grid Address:  http://hospitalnode.herokuapp.com/


In [16]:
worker.run_remote_inference(model_id="toy_model", data=th.tensor([1.0, 2]))

[0.7817554473876953]

### Disconnect from the network

In [17]:
grid_network.disconnect_nodes()